In [1]:
import gym
import numpy as np

from Layers import *
from Activations import *
from Losses import *
from Optimizers import *
from Models import *
from Metrics import *

In [2]:
# So Much Room for Activities
env = gym.make('CartPole-v1')

In [3]:
# Magic Numbers
discount_factor = 0.95
eps = 0.5
eps_decay_factor = 0.999
num_episodes=500

In [4]:
# Instantiate the model
model = GraphModel()

# ### Add layers
l1 = GraphDense(20, Relu())
l3 = GraphDense(2, Softmax())
l2 = GraphDense(2, Linear())
g = {
    l1: [],
    l2: [l1]
}
model.add(g, [l1], [l2], env.observation_space.shape[0]) 

# Set loss, optimizer and accuracy objects
model.set(
    loss=MeanSquaredError(),
    optimizer=GAdam(),
    accuracy=Accuracy_Regression()
)

# Finalize the model
model.finalize()


In [5]:
# Try, Try, and Try Again
for i in range(num_episodes):
    state, _ = env.reset()
    eps *= eps_decay_factor
    length = 0
    done = False
    
    while not done:
        length += 1
        
        if np.random.random() < eps:
            action = np.random.randint(0, 1)
        else:
            action = np.argmax(model.predict([state.reshape(-1, 4)])[0,0])
            
        new_state, reward, done, _, __ = env.step(action)
        if done or abs(new_state[2]) > abs(state[2]):
            reward = -1
        else:
            reward = 1
        
        target = reward + discount_factor * np.max(model.predict([new_state.reshape(-1, 4)]))
        target_vector = model.predict(state.reshape(-1, 4))
        target_vector[0, 0, action] = target
        
        model.train(state.reshape(-1, 4), 
                    target_vector.reshape(-1, 2), 
                    epochs=1, 
                    batch_size=1,
                    print_every=10)
        state = new_state
    
    print(i, ": ", length)

0 :  12
1 :  9
2 :  23
3 :  11
4 :  29
5 :  12
6 :  17
7 :  10
8 :  47
9 :  16
10 :  24
11 :  18
12 :  23
13 :  12
14 :  12
15 :  30
16 :  19
17 :  25
18 :  16
19 :  10
20 :  12
21 :  10
22 :  32
23 :  45
24 :  11
25 :  10
26 :  31
27 :  23
28 :  18
29 :  20
30 :  16
31 :  12
32 :  23
33 :  15
34 :  40
35 :  39
36 :  28
37 :  21
38 :  15
39 :  14
40 :  24
41 :  15
42 :  12
43 :  34
44 :  11
45 :  13
46 :  13
47 :  26
48 :  34
49 :  13
50 :  54
51 :  15
52 :  26
53 :  39
54 :  21
55 :  25
56 :  18
57 :  41
58 :  10
59 :  28
60 :  18
61 :  19
62 :  20
63 :  13
64 :  16
65 :  16
66 :  23
67 :  43
68 :  20
69 :  22
70 :  13
71 :  18
72 :  12
73 :  22
74 :  26
75 :  48
76 :  33
77 :  14
78 :  25
79 :  35
80 :  21
81 :  20
82 :  13
83 :  24
84 :  10
85 :  68
86 :  32
87 :  23
88 :  16
89 :  20
90 :  13
91 :  26
92 :  25
93 :  28
94 :  13
95 :  17
96 :  20
97 :  15
98 :  24
99 :  43
100 :  52
101 :  43
102 :  18
103 :  15
104 :  27
105 :  46
106 :  35
107 :  36
108 :  21
109 :  67
110 :  14
1

In [6]:
env.close()